In [ ]:
import os
import pygmt
import pandas as pd
import numpy as np

In [ ]:
# Creating the path for both the folders
datadir = "./"
path_plots = os.path.join(datadir, "Plots")
path_ev = os.path.join(datadir, "Events")
path_st = os.path.join(datadir, "Stations")

try: os.mkdir(path_plots)
except: print("The directory already exists")

In [ ]:
# Define your study site
minlon, maxlon = 90, 104
minlat, maxlat = 16, 30
lonc = (minlon + maxlon) / 2
latc = (minlat + maxlat) / 2

# Define stations provider
provider1 = "IRIS"
provider2 = "GFZ"

# Define etopo data file
topo_data = '@earth_relief_30s' #30 arc second global relief (SRTM15+V2.1 @ 1.0 km)

In [ ]:
# Defining the catalog duration
for i in range(22):
  year1 = 2000 + i
  year2 = 2001 + i
  
  print(f"For the Time-period: {year1}-{year2}")
  
  fig = pygmt.Figure()
  pygmt.config(MAP_FRAME_TYPE="fancy",MAP_DEGREE_SYMBOL="degree",MAP_FRAME_WIDTH="0.08c",MAP_FRAME_PEN="0.1p",MAP_TICK_PEN_PRIMARY="0.1p",MAP_TICK_LENGTH_PRIMARY="0.1c")
  pygmt.config(FORMAT_GEO_MAP="dddD")
  pygmt.config(FONT_LABEL="8p,4",MAP_LABEL_OFFSET="0.1c",FONT_ANNOT_PRIMARY="8p,4",FONT_TITLE="10p,4",MAP_TITLE_OFFSET="0.1c")

  fig.grdimage(region=[minlon, maxlon, minlat, maxlat], grid=topo_data, cmap="globe", shading=True,
      projection='M3i', frame=['xa4f2+l"longitude"', 'ya4f2+l"latitude"', 'WsNe'])
  fig.coast(frame=False,shorelines=True,borders='1/0.5p,black')

  # Plot stations
  try:
    df = pd.read_csv(f"{path_st}/stations_{provider1}_{year1}-{year2}.txt", sep='|', usecols=[0,1,4,5])
    df.columns = ["net","station","lat","lon"]
    fig.plot(x=df["lon"], y=df["lat"], no_clip=True, style="t0.2c", pen="0.5p", label=f"{provider1} Station")
  except:
    print(f"{provider1} doesn't have data for this particular region and duration.")
  
  try:
    df = pd.read_csv(f"{path_st}/stations_{provider2}_{year1}-{year2}.txt", sep='|', usecols=[0,1,4,5])
    df.columns = ["net","station","lat","lon"]
    fig.plot(x=df["lon"], y=df["lat"], no_clip=True, style="s0.2c", pen="0.5p", label=f"{provider2} Station")
  except:
    print(f"{provider2} doesn't have data for this particular region and duration.")

  fig.legend(position="jbl+o0.1c/0.1c", box="+gsnow")

  with fig.inset(position="jbl+w2.0c+o0.1c/0.8c"):
    fig.coast(
      region="g",
      projection=f"G{lonc}/{latc}/?",
      land="black",
      water="lightblue"
    )

    # Plot events
    df = pd.read_csv(f"{path_ev}/events_{year1}-{year2}.csv", sep=',', usecols=[0,1,2,3,5])
    df.columns = ["time","lat","lon","dep","mag"]
    fig.plot(x=df["lon"], y=df["lat"], color="brown", style="c0.02c")
    
    # Study area
    fig.plot(
      x = [minlon, minlon, maxlon, maxlon, minlon],
      y = [minlat, maxlat, maxlat, minlat, minlat],
      pen = "0.3p,red"
    )

  fig.savefig(f"{path_plots}/plot_{year1}_{year2}.png")
  fig.show()
    
  print()